## Finite Field

$$
p = 64 * 3 + 1
$$

In [91]:
F193.<a> = GF(193)
R193.<X> = F193[]
R193
Fp=F193
Fp

Finite Field of size 193

In [92]:
R_2.<X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2> = Fp[]; R_2

Multivariate Polynomial Ring in X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2 over Finite Field of size 193

In [93]:
from field import magic

Fp = magic(Fp)

In [94]:

def bits_le_with_width(i, width):
    if i >= 2**width:
        return "Failed"
    bits = []
    while width:
        bits.append(i % 2)
        i //= 2
        width -= 1
    return bits
        
bits_le_with_width(6, 5)

[0, 1, 1, 0, 0]

In [95]:
def is_power_of_two(n):
    d = n
    k = 0
    while d > 0:
        d >>= 1
        k += 1
    if n == 2**(k-1):
        return True
    else:
        return False

def next_power_of_two(n):
    assert n >= 0, "No negative integer"
    if is_power_of_two(n):
        return n
    d = n
    k = 0
    while d > 0:
        d >>= 1
        k += 1
    return k

is_power_of_two(15), next_power_of_two(15)

(False, 4)

In [96]:
(1 - X0) * (1 - X1)

X0*X1 - X0 - X1 + 1

In [97]:
Fp(((1 - X0) * (1 - X1))(X0=9, X1=19))

Field([144])

In [98]:
Fp((X0 * X1)(X0=9, X1=19))

Field([171])

## Foldable code 

In [99]:
# m: message
# k0: message length of base code
# c: blowup factor

def rep_encode(m, k0, c):
    assert len(m) % k0 == 0, "len(m): %d is not a multiple of k0: %d" % (len(m), k0)
    code = []
    for i in range(0, c*len(m), k0):
        for j in range(0, c):
            code += m[i:i+k0]
    return code

In [100]:
rep_encode([1,2],1, 3), rep_encode([1,2,3,4], 2, 3), rep_encode([1,2,3,4], 4, 3)

([1, 1, 1, 2, 2, 2],
 [1, 2, 1, 2, 1, 2, 3, 4, 3, 4, 3, 4],
 [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4])

### Reed-Solomon Code

Fix a sequence $\mathbf{\alpha} = (\alpha_1, \alpha_2, \ldots, \alpha_n)$ of $n$ distict elements from $\mathbf{F}_q$. Map a message $\mathbf{m} = (m_0, m_1, \ldots, m_{k_0 - 1})$ with $m_i \in \mathbf{F}_q$ to the degree $k_0 - 1$ polynomial.

$$
\mathbf{m} \mapsto f_{\mathbf{m}}(X)
$$

where

$$
f_{\mathbf{m}}(X) = \sum_{i = 0}^{k_0 - 1} m_i X^i
$$

Then

$$
\text{RS}_q[\mathbf{\alpha}, k_0](\mathbf{m}) = (f_{\mathbf{m}}(\alpha_1), f_{\mathbf{m}}(\alpha_2), \ldots, f_{\mathbf{m}}(\alpha_{n}))
$$

For simplicity, we can choose $\alpha_1 = 0, \alpha_2 = 1, \ldots, \alpha_{n} = n - 1$.

In [101]:
# Reed-Solomon code for m = (m_0, ..., m_{k_0 - 1})
# alpha: evaluation points
# c: blowup factor
def rs_encode_single(m, alpha, c):
    k0 = len(m)
    code = [None] * (k0 * c)
    for i in range(k0 * c): 
        # comput f_m(alpha[i])
        code[i] = sum(m[j] * (alpha[i] ** j) for j in range(k0))
    return code

# Reed-Solomon code for m
# m: message
# k0: message length of base code
# c: blowup factor
# default alpha: [0, 1, 2, ... , k0*c - 1]
def rs_encode(m, k0, c):
    assert len(m) % k0 == 0, "len(m): %d is not a multiple of k0: %d" % (len(m), k0)
    code = []
    alpha = list(range(k0 * c)) # 这里默认 alpha = [0, 1, 2, ... , k0*c - 1]
    for i in range(0, c*len(m), k0):
        code += rs_encode_single(m[i:i+k0], alpha, c)
    return code

In [102]:
rs_encode([1,2],1,3), rs_encode([1,2,3,4], 2, 3), rs_encode([1,2,3,4], 4, 3)

([1, 1, 1, 2, 2, 2],
 [1, 3, 5, 7, 9, 11, 3, 7, 11, 15, 19, 23],
 [1, 10, 49, 142, 313, 586, 985, 1534, 2257, 3178, 4321, 5710])

In [103]:
# m: message with length `n`
# k0: message length of code `G_0`,  G_0: k0 \otimes ck0
# depth: 
# c: blowup factor
# G_0: an encoding function
#      G_0(m, k0, c)
# T: RM code table

def basefold_encode(m, k0, depth, c, T, G0=rep_encode, debug=False):
    if debug: print(">>> basefold_encode: m={}, k0={}, d={}, blowup_factor={}, T={}".format(m, k0, depth, c, T))
    kd = k0 * 2^depth
    blowup_factor = c
    assert len(m) == kd, "len(m): %d != kd: %d" % (len(m), kd)
    assert len(T) == depth, "len(T): %d != depth: %d" % (len(T), depth)
     
    chunk_size = k0
    chunk_num = 2^depth
    
    code = []
    for i in range(chunk_num):
        chunk = m[i*chunk_size: (i+1)*chunk_size]
        chunk_code = G0(chunk, chunk_size, blowup_factor)
        code += chunk_code
    if debug: print(">>> basefold_encode: code=", code)

    if depth == 0:
        return code

    chunk_size = k0 * blowup_factor
    if debug: print(">>> basefold_encode: chunk_size=", chunk_size)
    if debug: print(">>> basefold_encode: chunk_num=", chunk_num)


    for i in range(0, depth):
        table = T[i]
        assert len(table) == chunk_size, "table[{}] != chunk_size, len(table)={}, chunk_size={})".format(i, len(table), chunk_size)
        if debug: print(">>> basefold_encode: table=", table)
        for c in range(0, chunk_num, 2):
            left  = code[    c*chunk_size : (c+1)*chunk_size]
            right = code[(c+1)*chunk_size : (c+2)*chunk_size]
            if debug: print(">>> basefold_encode: left={}, right={}".format(left, right))

            for j in range(0, chunk_size):
                if debug: print(">>> basefold_encode: i={}, c={}, j={}".format(i, c, j))
                rhs = right[j] * table[j]
                lhs = left[j]
                code[(c)*chunk_size + j] = lhs + rhs
                code[(c+1)*chunk_size + j] = lhs - rhs
        chunk_size = chunk_size * 2
        chunk_num = chunk_num // 2
    return code

In [104]:
# basefold_encode(m, k0, depth, c, G0, T, debug=False):

def basefold_encode_rec(m, k0, depth, c, T, G0=rep_encode, debug=False):
    if depth == 0:
        assert len(m) == k0, "len(m): %d != k0: %d" % (len(m), k0)
        return G0(m, k0, c)
    
    kd = len(m)
    half = kd / 2
    code_size = c * kd

    m_left = m[:half]
    m_right = m[half:]
    
    code_left = basefold_encode_rec(m_left, k0, depth-1, c, T, G0=G0, debug=debug)
    print("code_left=", code_left)
    code_right = basefold_encode_rec(m_right, k0, depth-1, c, T, G0=G0, debug=debug)
    print("code_right=", code_right)

    t = T[depth-1]
    assert len(t) == code_size/2, "wrong table size, len(table)={}, code_size={})".format(len(t), code_size/2)
    if debug: print("t=", t)
    code_half = kd * c // 2
    code = [0 for _ in range(code_half * 2)]
    for j in range(code_half):
        code[j] = code_left[j] + t[j] * code_right[j]
        code[j+code_half] = code_left[j] - t[j] * code_right[j]
    return code

In [105]:
basefold_encode([1, 2], k0=1, depth=1, c=2, T=[[X0, X1]], debug=True)

>>> basefold_encode: m=[1, 2], k0=1, d=1, blowup_factor=2, T=[[X0, X1]]
>>> basefold_encode: code= [1, 1, 2, 2]
>>> basefold_encode: chunk_size= 2
>>> basefold_encode: chunk_num= 2
>>> basefold_encode: table= [X0, X1]
>>> basefold_encode: left=[1, 1], right=[2, 2]
>>> basefold_encode: i=0, c=0, j=0
>>> basefold_encode: i=0, c=0, j=1


[2*X0 + 1, 2*X1 + 1, -2*X0 + 1, -2*X1 + 1]

In [106]:
basefold_encode_rec([1, 2], k0=1, depth=1, c=2, T=[[X0, X1]])

code_left= [1, 1]
code_right= [2, 2]


[2*X0 + 1, 2*X1 + 1, -2*X0 + 1, -2*X1 + 1]

In [107]:
basefold_encode([1, 2], k0=1, depth=1, c=3, T=[[X0, X1, X2]], debug=True)

>>> basefold_encode: m=[1, 2], k0=1, d=1, blowup_factor=3, T=[[X0, X1, X2]]
>>> basefold_encode: code= [1, 1, 1, 2, 2, 2]
>>> basefold_encode: chunk_size= 3
>>> basefold_encode: chunk_num= 2
>>> basefold_encode: table= [X0, X1, X2]
>>> basefold_encode: left=[1, 1, 1], right=[2, 2, 2]
>>> basefold_encode: i=0, c=0, j=0
>>> basefold_encode: i=0, c=0, j=1
>>> basefold_encode: i=0, c=0, j=2


[2*X0 + 1, 2*X1 + 1, 2*X2 + 1, -2*X0 + 1, -2*X1 + 1, -2*X2 + 1]

In [108]:
basefold_encode([1,2,3,4], k0=1, depth=2, c=3, T=[[X0, X1, X2], [Y0, Y1, Y2, Y3, Y4, Y5]])

[4*X0*Y0 + 2*X0 + 3*Y0 + 1,
 4*X1*Y1 + 2*X1 + 3*Y1 + 1,
 4*X2*Y2 + 2*X2 + 3*Y2 + 1,
 -4*X0*Y3 - 2*X0 + 3*Y3 + 1,
 -4*X1*Y4 - 2*X1 + 3*Y4 + 1,
 -4*X2*Y5 - 2*X2 + 3*Y5 + 1,
 -4*X0*Y0 + 2*X0 - 3*Y0 + 1,
 -4*X1*Y1 + 2*X1 - 3*Y1 + 1,
 -4*X2*Y2 + 2*X2 - 3*Y2 + 1,
 4*X0*Y3 - 2*X0 - 3*Y3 + 1,
 4*X1*Y4 - 2*X1 - 3*Y4 + 1,
 4*X2*Y5 - 2*X2 - 3*Y5 + 1]

## Basefold IOPP

$$
B^{(monomial)} = (1, \alpha_0)\otimes (1, \alpha_1) \otimes \cdots \otimes (1, \alpha_{k-1})
$$

$$
B^{(multilinear)} = ((1-\alpha_0), \alpha_0)\otimes ((1-\alpha_1), \alpha_1) \otimes \cdots \otimes ((1-\alpha_{k-1}), \alpha_{k-1})
$$

In [109]:
def basefold_fri_monomial_basis(vs, table, alpha, debug=False):
    assert len(table) == len(vs)/2, "len(table) is not double len(vs), len(table) = %d, len(vs) = %d" % (len(table), len(vs))
    n = len(vs)
    half = n / 2
    new_vs = []
    left = vs[:half]
    right = vs[half:]

    for i in range(0, half):
        if debug: print("(left[i] + right[i])/2=", (left[i] + right[i])/2)
        if debug: print("(left[i] + right[i])/2=", (left[i] + right[i])/2)
        new_vs.append((left[i] + right[i])/2 + (alpha) * (left[i] - right[i])/(2*table[i]))
    return new_vs

In [110]:
def basefold_fri_multilinear_basis(vs, table, c, debug=False):
    assert len(table) == len(vs)/2, "len(table) is not double len(vs), len(table) = %d, len(vs) = %d" % (len(table), len(vs))
    n = len(vs)
    half = n / 2
    new_vs = []
    left = vs[:half]
    right = vs[half:]

    for i in range(0, half):
        if debug: print("(left[i] + right[i])/2=", (left[i] + right[i])/2)
        if debug: print("(left[i] + right[i])/2=", (left[i] + right[i])/2)
        new_vs.append((1 - c) * (left[i] + right[i])/2 + (c) * (left[i] - right[i])/(2*table[i]))
    return new_vs

In [111]:
pi_0 = basefold_encode([1, 2, 3, 4], k0=1, depth=2, c=2, T=[[X4, X5], [X0, X1, X2, X3]]); pi_0

[4*X0*X4 + 3*X0 + 2*X4 + 1,
 4*X1*X5 + 3*X1 + 2*X5 + 1,
 -4*X2*X4 + 3*X2 - 2*X4 + 1,
 -4*X3*X5 + 3*X3 - 2*X5 + 1,
 -4*X0*X4 - 3*X0 + 2*X4 + 1,
 -4*X1*X5 - 3*X1 + 2*X5 + 1,
 4*X2*X4 - 3*X2 - 2*X4 + 1,
 4*X3*X5 - 3*X3 - 2*X5 + 1]

In [112]:
pi_1 = basefold_fri_multilinear_basis(pi_0, [X0, X1, X2, X3], A0); pi_1

[2*X4*A0 + 2*X4 + 2*A0 + 1,
 2*X5*A0 + 2*X5 + 2*A0 + 1,
 -2*X4*A0 - 2*X4 + 2*A0 + 1,
 -2*X5*A0 - 2*X5 + 2*A0 + 1]

In [113]:
pi_2 = basefold_fri_multilinear_basis(pi_1, [X4, X5], A1); pi_2

[2*A0 + A1 + 1, 2*A0 + A1 + 1]

## Basefold Evaluation Argument

$$
v = f(\mathbf{z}) = \sum_{\mathbf{b}}f(\mathbf{b})\cdot \tilde{eq}_{\mathbf{z}}(\mathbf{b})
$$

by sumcheck, the statement can be reduced to

$$
v' \overset{?}{=} f(\mathbf{r})\cdot \tilde{eq}_{\mathbf{z}}(\mathbf{r})
$$

and we can have the following statement, since the code is linear
$$
\mathsf{encode}(v'/\tilde{eq}_{\mathbf{z}}(\mathbf{r})) \overset{?}{=} \mathsf{encode}\big(f(\mathbf{r})\big)
$$

Fortunately, the right hand side is exactly the result of FRI-folding, and the left hand side can be computed by verifier himself.

### More features

- Add RS code support for `G0`

### TODO

- Change Rnd into a transcript that absorbs messages and squeezes challenges.
- Change `Proof` into a struct, dict or class
- Add inputs validation
- Add error messages for `assert`


In [114]:
from mle2 import MLEPolynomial
from merkle import MerkleTree

def query_phase(transcript, first_tree: MerkleTree, first_oracle, trees: list, oracles: list, num_vars, num_verifier_queries, debug=False):
    queries = [transcript.squeeze_byte_challenge() % num_vars for _ in range(num_verifier_queries)]

    query_paths = []
    # query paths
    for q in queries:
        num_vars_copy = num_vars
        cur_path = []
        indices = []
        x0 = q
        x1 = q - num_vars_copy / 2 if q >= num_vars_copy / 2 else q + num_vars_copy / 2
        if x1 < x0:
            x0, x1 = x1, x0
        
        cur_path.append((first_oracle[x0], first_oracle[x1]))
        indices.append(x0)
        q = x0
        num_vars_copy >>= 1

        for oracle in oracles:
            x0 = q
            x1 = q - num_vars_copy / 2 if q >= num_vars_copy / 2 else q + num_vars_copy / 2
            if x1 < x0:
                x0, x1 = x1, x0
            
            cur_path.append((oracle[x0], oracle[x1]))
            if debug: print("x0:", x0, "x1:", x1, "num_vars:", num_vars_copy)
            if debug: print("oracle:", oracle)
            indices.append(x0)
            q = x0
            num_vars_copy >>= 1
        
        query_paths.append((cur_path, indices))

    # merkle paths
    merkle_paths = []
    for _, indices in query_paths:
        cur_query_paths = []
        for i, idx in enumerate(indices):
            if i == 0:
                cur_query_paths.append(first_tree.get_authentication_path(idx))
                if debug: print("mp:", cur_query_paths[-1])
                if debug: print("commit:", first_tree.root)
                if debug: print("idx:", idx)
            else:
                cur_tree = trees[i - 1]
                assert isinstance(cur_tree, MerkleTree)
                cur_query_paths.append(cur_tree.get_authentication_path(idx))
                if debug: print("mp:", cur_query_paths[-1])
                if debug: print("commit:", first_tree.root)
                if debug: print("idx:", idx)
        merkle_paths.append(cur_query_paths)

    return query_paths, merkle_paths

In [115]:
from unipolynomial import UniPolynomial

# Define prove
# f_code: Encode(f)
# f_evals: evaluations of f
# us: point
# v: f(point) = v
# k: number of variables, k = len(u)
# T: random code table
# blowup_factor: as named
# transcript: as named

def prove_basefold_evaluation_arg_multilinear_basis(f_code, f_evals, us, v, k, T, blowup_factor, commit, num_verifier_queries, transcript, debug=False):
    # TODO: check if the length of f is a power of 2
    
    n = len(f_evals)
    half = n >> 1
    assert len(T) == k, "wrong table size, k={}, len(T)={}".format(k, len(T))
    f_code_copy = f_code[:]
    f = f_evals[:]
    eq = MLEPolynomial.eqs_over_hypercube(us)
    
    challenge_vec = []
    sumcheck_sum = v
    h_poly_vec = []
    f_code_vec = []
    f_code_trees = []
    for i in range(k):
        if debug: print("sumcheck round {}".format(i))
        f_low = f[:half]
        f_high = f[half:]
        eq_low = eq[:half]
        eq_high = eq[half:]
        
        # print("low={}, high={}".format(low, high))

        h_eval_at_0 = sum([f_low[j] * eq_low[j] for j in range(half)])
        h_eval_at_1 = sum([f_high[j] * eq_high[j] for j in range(half)])
        h_eval_at_2 = sum([ (2 * f_high[j] - f_low[j]) * (2 * eq_high[j] - eq_low[j]) for j in range(half)])
        h_poly_vec.append([h_eval_at_0, h_eval_at_1, h_eval_at_2])
        
        if debug: print("> sumcheck: h=[{},{},{}]".format(h_eval_at_0, h_eval_at_1, h_eval_at_2))
        if debug: print("> sumcheck: {} ?= {}".format(h_eval_at_0 + h_eval_at_1, sumcheck_sum))
        
        assert h_eval_at_0 + h_eval_at_1 == sumcheck_sum, "{} + {} != {}" % (h_eval_at_0, h_eval_at_1, sumcheck_sum)

        # Receive a random number from the verifier
        alpha = A0 * transcript.squeeze_byte_challenge()
        
        challenge_vec.append(alpha)

        # fold(low, high, alpha)
        f = [(1 - alpha) * f_low[i] + alpha * f_high[i] for i in range(half)]
        eq = [(1 - alpha) * eq_low[i] + alpha * eq_high[i] for i in range(half)]
        if debug: print("> sumcheck: f_folded = {}".format(f))
        if debug: print("> sumcheck: eq_folded = {}".format(eq))

        # compute the new sum = h(alpha)
        sumcheck_sum = UniPolynomial.uni_eval_from_evals([h_eval_at_0, h_eval_at_1, h_eval_at_2], alpha, [Fp(0),Fp(1),Fp(2)])
        if debug: print("> sumcheck: sumcheck_sum = {}".format(sumcheck_sum))

        if debug: print("fri round {}".format(i))

        # # Basefold fri
        f_code = basefold_fri_multilinear_basis(f_code, T[k-i-1], alpha, debug=debug)
        if debug: print("> fri: f_code_folded=", f_code)
        f_code_vec.append(f_code)
        f_code_trees.append(MerkleTree(f_code))

        # DEBUG: consistency check (invariant)
        # Enc(fold(message)) = fold(Enc(message)) 
        debug_f_folded_code = basefold_encode(m=f, k0=1, depth=k-i-1, c=blowup_factor, G0=rs_encode, T=T[:k-i-1], debug=debug)
        if debug: print("> fri: enc({})={}".format(f, debug_f_folded_code))
        assert f_code == debug_f_folded_code, "Enc(fold(message)) != fold(Enc(message))"

        transcript.write_field_elements([h_eval_at_0, h_eval_at_1, h_eval_at_2])
        transcript.write_field_elements(f_code)
        
        half = half >> 1

    # DEBUG:
    f_eval_at_random = sumcheck_sum/eq[0]
    assert rs_encode([f_eval_at_random], k0=1, c=blowup_factor) == f_code, "❌: Encode(f(rs)) != f_code_0"
    if debug: print("end: f_code={}, sum={}, sum/eq={}".format(f_code, sumcheck_sum, sumcheck_sum/eq[0]))
    if debug: print("🙈: fold(f_code) == encode(fold(f_eq)/fold(eq(us)))")

    query_paths, merkle_paths = query_phase(transcript, commit, f_code_copy, f_code_trees, f_code_vec, len(f_code_copy), num_verifier_queries, debug)

    # return (h_poly_vec, challenge_vec, f_code_vec)
    return {
        'h_poly_vec': h_poly_vec,
        'f_code_vec': f_code_vec,
        'challenge_vec': challenge_vec,
        'f_code_trees': f_code_trees,
        'query_paths': query_paths,
        'merkle_paths': merkle_paths
    }

In [116]:
from merkle import verify_decommitment
from transcript import Transcript

def verify_queries(commit, proof, k, num_vars, num_verifier_queries, T, debug=False):
    transcript = Transcript()
    transcript.write_field_elements(commit.root)

    fold_challenges = []
    for i in range(k):
        fold_challenges.append(A0 * transcript.squeeze_byte_challenge())
        transcript.write_field_elements(proof['h_poly_vec'][i])
        transcript.write_field_elements(proof['f_code_vec'][i])

    queries = [transcript.squeeze_byte_challenge() % num_vars for _ in range(num_verifier_queries)]
    # query loop
    for q, (cur_path, _), mps in zip(queries, proof['query_paths'], proof['merkle_paths']):
        if debug: print("cur_path:", cur_path)
        num_vars_copy = num_vars
        # fold loop
        for i, mp in enumerate(mps):
            x0 = q
            x1 = q - num_vars_copy / 2 if q >= num_vars_copy / 2 else q + num_vars_copy / 2
            if x1 < x0:
                x0, x1 = x1, x0
                
            code_left, code_right = cur_path[i][0], cur_path[i][1]

            if i != len(mps) - 1:
                table = T[k-i-1]
                if debug: print("table:", table)
                if debug: print("x0:", x0)
                if debug: print("x1:", x1)
                f_code_folded = cur_path[i + 1][0 if x0 < num_vars_copy / 4 else 1]
                alpha = fold_challenges[i]
                if debug: print("f_code_folded:", f_code_folded)
                if debug: print("expected:", ((1 - alpha) * (code_left + code_right)/2 + (alpha) * (code_left - code_right)/(2*table[x0])))
                if debug: print("code_left:", code_left)
                if debug: print("code_right:", code_right)
                if debug: print("alpha:", alpha)
                assert f_code_folded == ((1 - alpha) * (code_left + code_right)/2 + (alpha) * (code_left - code_right)/(2*table[x0])), "failed to check multilinear base fri"

            if i == 0:
                if debug: print("mp:", mp)
                if debug: print("commit:", commit.root)
                if debug: print("idx:", x0)
                assert verify_decommitment(x0, code_left, mp, commit.root)
            else:
                if debug: print("mp:", mp)
                if debug: print("commit:", proof['f_code_trees'][i - 1].root)
                if debug: print("idx:", x0)
                assert verify_decommitment(x0, code_left, mp, proof['f_code_trees'][i - 1].root)

            num_vars_copy >>= 1
            q = x0

In [117]:
def verify_basefold_evaluation_arg_multilinear_basis(f_code, commit, proof, us, v, d, k, T, blowup_factor, num_verifier_queries, Rng, debug=False):
    # TODO: check if the length of f is a power of 2
    
    N = len(f_code)
    assert k == len(us), "k != len(us), k = %d, len(us) = %d" % (k, len(us))
    n = 1 << k
    assert N == n * blowup_factor, "N != n * blowup_factor, N = %d, n = %d, blowup_factor = %d" % (N, n, blowup_factor)
    
    h_poly_vec = proof['h_poly_vec']
    challenge_vec = proof['challenge_vec']
    f_code_vec = proof['f_code_vec']
    sumcheck_sum = v
    half = n >> 1
    f_last_code = f_code
    eq_evals = MLEPolynomial.eqs_over_hypercube(us)
    
    for i in range(k):
        if debug: print("sumcheck round {}".format(i))
        h_evals = h_poly_vec[i]
        assert d+1 == len(h_evals), "d+1 != len(h_evals), d+1 = %d, len(h_evals) = %d" % (d+1, len(h_evals))

        assert sumcheck_sum == h_evals[0] + h_evals[1], "sumcheck_sum != h_evals[0] + h_evals[1], sumcheck_sum = %d, h_evals[0] = %d, h_evals[1] = %d" % (sumcheck_sum, h_evals[0], h_evals[1])
        # print("low={}, high={}".format(low, high))

        alpha = challenge_vec[i]

        sumcheck_sum = UniPolynomial.uni_eval_from_evals(h_evals, alpha, [Fp(0),Fp(1),Fp(2)])

        eq_low = eq_evals[:half]
        eq_high = eq_evals[half:]
        if debug: print("eq_low={}, eq_high={}".format(eq_low, eq_high))
        eq_evals = [(1-alpha) * eq_low[i] + alpha * eq_high[i] for i in range(half)]

        if debug: print("fri round {}".format(i))

        table = T[k-i-1]
        f_code_folded = f_code_vec[i]
        assert len(f_code_folded) == half * blowup_factor, "len(f_code_folded) != half * blowup_factor, len(f_code_folded) = %d, half = %d, blowup_factor = %d" % (len(f_code_folded), half, blowup_factor)
        code_left = f_last_code[:half*blowup_factor]
        code_right = f_last_code[half*blowup_factor:]
        if debug: print("code_left={}".format(code_left))
        if debug: print("code_right={}".format(code_right))
        if debug: print("f_code_folded={}".format(f_code_folded))
        if debug: print("alpha={}".format(alpha))
        if debug: print("table={}".format(table))
        for j in range(half*blowup_factor):
            assert f_code_folded[j] == ((1 - alpha) * (code_left[j] + code_right[j])/2 + (alpha) * (code_left[j] - code_right[j])/(2*table[j])), "failed to check multilinear base fri"
        f_last_code = f_code_folded
        half = half >> 1

    verify_queries(commit, proof, k, N, num_verifier_queries, T, debug)
        
    # check the final code
    final_code = f_code_vec[i]
    assert len(final_code) == blowup_factor, "len(final_code) != blowup_factor, len(final_code) = %d, blowup_factor = %d" % (len(final_code), blowup_factor)
    for i in range(len(final_code)):
        assert final_code[0] == final_code[i], "final_code is not a repetition code"
    # check f(alpha_vec)
    f_eval_at_random = sumcheck_sum/eq_evals[0]
    if debug: print("f_eval_at_random={}".format(f_eval_at_random))
    if debug: print("rs_encode([f_eval_at_random], k0=1, c=blowup_factor)=", rs_encode([f_eval_at_random], k0=1, c=blowup_factor))
    assert rs_encode([f_eval_at_random], k0=1, c=blowup_factor) == f_code_folded, "❌: Encode(f(rs)) != f_code_0"
    print("✅: Verified! fold({}) == encode(fold(f_eq)/fold(eq(us)))".format(f_code))

    return True

In [118]:
ff = [Fp(1),Fp(2),Fp(3),Fp(4)]
ff_code = basefold_encode(m=ff, k0=1, depth=2, c=2, G0=rs_encode, T=[[X4, X5], [X0, X1, X2, X3]]); ff_code

[Field([4*X0*X4 + 3*X0 + 2*X4 + 1]),
 Field([4*X1*X5 + 3*X1 + 2*X5 + 1]),
 Field([-4*X2*X4 + 3*X2 - 2*X4 + 1]),
 Field([-4*X3*X5 + 3*X3 - 2*X5 + 1]),
 Field([-4*X0*X4 - 3*X0 + 2*X4 + 1]),
 Field([-4*X1*X5 - 3*X1 + 2*X5 + 1]),
 Field([4*X2*X4 - 3*X2 - 2*X4 + 1]),
 Field([4*X3*X5 - 3*X3 - 2*X5 + 1])]

In [119]:
commit = MerkleTree(ff_code)
commit.root

'd7732ef5605e991b2909cb0219410cb35b49f23df6d5c909ecfe2ebacf64ce6d'

In [120]:
from transcript import Transcript

transcript = Transcript()
transcript.write_field_elements(commit.root)
proof = prove_basefold_evaluation_arg_multilinear_basis(ff_code, ff, [3,4], 12, 2, [[X4, X5], [X0, X1, X2, X3]], 2, commit, 4, transcript); proof

{'h_poly_vec': [[Field([181]), Field([24]), Field([88])],
  [Field([72*A0^2 + 10*A0 + 6]),
   Field([85*A0^2 + 73*A0 - 18]),
   Field([-95*A0^2 - 21*A0 - 72])]],
 'f_code_vec': [[Field([-10*X4*A0 + 2*X4 - 10*A0 + 1]),
   Field([-10*X5*A0 + 2*X5 - 10*A0 + 1]),
   Field([10*X4*A0 - 2*X4 - 10*A0 + 1]),
   Field([10*X5*A0 - 2*X5 - 10*A0 + 1])],
  [Field([89*A0 + 1]), Field([89*A0 + 1])]],
 'challenge_vec': [-5*A0, -94*A0],
 'f_code_trees': [<merkle.MerkleTree object at 0x12cf61410>,
 'query_paths': [([(Field([4*X1*X5 + 3*X1 + 2*X5 + 1]),
     Field([-4*X1*X5 - 3*X1 + 2*X5 + 1])),
    (Field([-10*X5*A0 + 2*X5 - 10*A0 + 1]),
     Field([10*X5*A0 - 2*X5 - 10*A0 + 1])),
    (Field([89*A0 + 1]), Field([89*A0 + 1]))],
   [1, 1, 0]),
  ([(Field([-4*X3*X5 + 3*X3 - 2*X5 + 1]), Field([4*X3*X5 - 3*X3 - 2*X5 + 1])),
    (Field([-10*X5*A0 + 2*X5 - 10*A0 + 1]),
     Field([10*X5*A0 - 2*X5 - 10*A0 + 1])),
    (Field([89*A0 + 1]), Field([89*A0 + 1]))],
   [3, 1, 0]),
  ([(Field([4*X0*X4 + 3*X0 + 2*X4 + 1]

In [121]:
# verify_basefold_evaluation_arg_multilinear_basis(f_code, arg, us, v, d, k, T, blowup_factor, Rng):

verify_basefold_evaluation_arg_multilinear_basis(ff_code, commit, proof, us=[3,4], v=12, d=2, k=2, T=[[X4, X5], [X0, X1, X2, X3]], blowup_factor=2, num_verifier_queries=4, Rng=[A1, A0])

✅: Verified! fold([Field([4*X0*X4 + 3*X0 + 2*X4 + 1]), Field([4*X1*X5 + 3*X1 + 2*X5 + 1]), Field([-4*X2*X4 + 3*X2 - 2*X4 + 1]), Field([-4*X3*X5 + 3*X3 - 2*X5 + 1]), Field([-4*X0*X4 - 3*X0 + 2*X4 + 1]), Field([-4*X1*X5 - 3*X1 + 2*X5 + 1]), Field([4*X2*X4 - 3*X2 - 2*X4 + 1]), Field([4*X3*X5 - 3*X3 - 2*X5 + 1])]) == encode(fold(f_eq)/fold(eq(us)))


True

In [122]:
from field import Field

print(Field.get_operation_count())
Field.reset_operation_count()
print(Field.get_operation_count())

{'add': 174, 'sub': 97, 'mul': 202, 'div': 70}
{'add': 0, 'sub': 0, 'mul': 0, 'div': 0}


In [123]:
print(Fp.Fp)
print(Fp.zero())
print(Fp.one())
print(Fp.primitive_element())

Finite Field of size 193
0
1
5
